This notebook is a mess, it's just provided if you want to see how I scraped the data

In [1]:
%pylab inline
import requests
import json
from tqdm import tqdm
import os
import time

Populating the interactive namespace from numpy and matplotlib


# get sales

In [2]:
sales_data_file = '.cache/sales.json'
try:
    sales = json.load(open(sales_data_file))
except FileNotFoundError:
    sales = {}
len(sales)

4

In [3]:
if len(sales)==0:
    r = requests.get("https://kittysales.herokuapp.com/data?offset=0&count=200000")
    sales = r.json()
    json.dump(sales, open(sales_data_file,'w'))
len(sales)

4

# get stats

In [4]:
api_data_file = '.cache/api_data.json'
try:
    api_data = json.load(open(api_data_file))
except FileNotFoundError:
    api_data = {}
len(api_data)

10024

In [5]:
wait=1 # exponential backoff
wait_mult = 1.01
keys_left = set(range(1, 100000)) - set([int(i) for i in api_data.keys()])
for i in tqdm(keys_left):
    
    if i in api_data:
        continue
    i = str(i)
    if i in api_data:
        continue
    url = 'http://api.cryptokitties.co/kitties/%s'%i
    try:
        r = requests.get(url)
    except Exception as e:
        wait*=wait_mult
        print('error1', i, e)
        time.sleep(wait)
    
    if 'Not Found' in r.content.decode():
        print(i, r.text)
        api_data[i]=np.nan
        continue
    
    try:
        api_data[i] = r.json()
    except Exception as e:
        print('error2', i, r, e)
        wait*=wait_mult
        time.sleep(wait)

  0%|          | 19/89976 [00:43<57:44:01,  2.31s/it]

error2 10043 <Response [503]> Expecting value: line 1 column 1 (char 0)


  0%|          | 20/89976 [00:54<68:03:33,  2.72s/it]

error2 10044 <Response [503]> Expecting value: line 1 column 1 (char 0)


  2%|▏         | 1491/89976 [18:43<18:31:08,  1.33it/s]

error2 11515 <Response [503]> Expecting value: line 1 column 1 (char 0)


  2%|▏         | 1881/89976 [24:22<19:01:32,  1.29it/s]

KeyboardInterrupt: 

In [8]:
json.dump(api_data, open(api_data_file,'w'))
len(api_data)

10024

In [2]:
url = 'http://api.cryptokitties.co/kitties?limit=100&offset=%s'%0
r = requests.get(url)
kitty_data=r.json()
total_kitties = kitty_data['total']
limit=20
overlap=3 # total keeps changing so lets have some overlap to ensure we get them all
api_data2={}

In [3]:
api_data2_file = '.cache/api_data2.json'
try:
    api_data2 = json.load(open(api_data2_file))
except FileNotFoundError:
    api_data2 = {}
len(api_data2)

2142

In [4]:
url = 'http://api.cryptokitties.co/kitties'
r = requests.get(url)
kitty_data = r.json()
total_kitties = kitty_data['total']
total_kitties

165164

In [ ]:
for i in tqdm(list(range(limit-overlap, total_kitties + limit, limit-overlap))):
    offset = kitty_data['total'] - i
    url = 'http://api.cryptokitties.co/kitties?limit=%s&offset=%s'%(limit, offset)
    
    r = requests.get(url)
    
    try:
        kitty_data = r.json()
    except json.JSONDecodeError as e:
        print('error', offset, e)        
        continue
        
    print(kitty_data['total'], kitty_data['limit'], i, offset, [d['id'] for d in kitty_data['kitties']])
    for kitty in kitty_data['kitties']:
        if kitty['id'] not in api_data2:
            api_data2[kitty['id']] = kitty 

  0%|          | 1/9716 [00:04<12:17:46,  4.56s/it]

165164 20 17 165147 [17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


  0%|          | 2/9716 [00:08<11:40:36,  4.33s/it]

165164 20 34 165130 [34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15]


  0%|          | 3/9716 [00:13<11:57:20,  4.43s/it]

...

167988 20 36193 131795 [37814, 37813, 37812, 37811, 37810, 37809, 37808, 37807, 37806, 37805, 37804, 37803, 37802, 37801, 37800, 37799, 37798, 37797, 37796, 37795]


 22%|██▏       | 2130/9716 [2:06:17<7:29:47,  3.56s/it]

167998 20 36210 131778 [37834, 37833, 37832, 37831, 37830, 37829, 37828, 37827, 37826, 37825, 37824, 37823, 37822, 37821, 37820, 37819, 37818, 37817, 37816, 37815]


 22%|██▏       | 2131/9716 [2:06:20<7:29:42,  3.56s/it]

167998 20 36227 131771 [37848, 37847, 37846, 37845, 37844, 37843, 37842, 37841, 37840, 37839, 37838, 37837, 37836, 37835, 37834, 37833, 37832, 37831, 37830, 37829]


In [48]:
len(api_data2)

2142

In [32]:
i=200
offset = total_kitties - i
url = 'http://api.cryptokitties.co/kitties?limit=%s&offset=%s'%(limit, offset)

r = requests.get(url)

kitty_data = r.json()

print(kitty_data['total'], i, offset, [d['id'] for d in kitty_data['kitties']])

164438 200 164238 [1104, 1103, 1102, 1101, 1100, 1099, 1098, 1097, 1096, 1095, 1094, 1093, 1092, 1091, 1090, 1089, 1088, 1087, 1086, 1085]


In [37]:
i=201
offset = total_kitties - i
url = 'http://api.cryptokitties.co/kitties?limit=%s&offset=%s'%(limit, offset)

r = requests.get(url)

kitty_data = r.json()

print(kitty_data['total'], i, offset, [d['id'] for d in kitty_data['kitties']])

164470 201 164237 [165991, 165990, 165989, 165988, 165987, 165986, 165985, 165984, 165983, 165982, 165981, 165980, 165979, 165978, 165977, 165976, 165975, 165974, 165973, 165972]


In [17]:
kitty_data['limit'], url

(20, 'http://api.cryptokitties.co/kitties?limit=100&offset=164109')

In [30]:
json.dump(api_data2, open(api_data2_file,'w'))
len(api_data2)

2142

In [15]:
offset = kitty_data['total'] - i
url = 'http://api.cryptokitties.co/kitties?limit=%s&offset=%s'%(limit, offset)
r = requests.get(url)
kitty_data = r.json()
[d['id'] for d in kitty_data['kitties']]

[165691,
 165690,
 165689,
 165688,
 165687,
 165686,
 165685,
 165684,
 165683,
 165682,
 165681,
 165680,
 165679,
 165678,
 165677,
 165676,
 165675,
 165674,
 165673,
 165672]

In [22]:
url = 'http://api.cryptokitties.co/kitties?limit=%s&offset=%s'%(limit, 163783-)
r = requests.get(url)
kitty_data = r.json()
[d['id'] for d in kitty_data['kitties']]

[1218,
 1217,
 1216,
 1215,
 1214,
 1213,
 1212,
 1211,
 1210,
 1209,
 1208,
 1207,
 1206,
 1205,
 1204,
 1203,
 1202,
 1201,
 1200,
 1199]

164096

# genetics


```js
// go here https://etherscan.io/token/0x06012c8cf97bead5deae237070f9587f8e7a266d#readContract
// breakpoint at readContract 126 (where is says `eval(functionNametoCall + "(" + strparas + ");");   `)
// then use the gui to use getKitty
// when it breaks execute this code
data={}

for (i = 0; i < 100000; i++) {
    if (i%10==0) {console.log(i, new Date())}
    if (data[i]) {continue}
    try {
        d = myContractInstance.getKitty(''+i);

        // convert from bignumber
        data[i] = d.map(dd=>dd.toFixed? dd.toFixed(): dd);
    } catch (e ) {
        console.log(i,'failed',e)
    }
}
copy(data)
```

In [9]:

genetics_file = '.cache/genes.json'
genetics = json.load(open(genetics_file))
len(genetics)

38422

# Transform

Lets embedd the genetics

In [46]:
# isGestating, isReady, cooldownindex, nextActionAt, siringWithId, birthTime, sireId, generation, genes
genes = genetics['0'][-1]
def longstr_to_genes(genes):
    return [int(i)/9 for i in genes]
genes = longstr_to_genes(genes)
genes

[0.1111111111111111,
 0.1111111111111111,
 0.5555555555555556,
 0.7777777777777778,
 1.0,
 0.2222222222222222,
 0.0,
 0.8888888888888888,
 1.0,
 0.2222222222222222,
 0.3333333333333333,
 0.7777777777777778,
 0.3333333333333333,
 0.1111111111111111,
 0.6666666666666666,
 0.1111111111111111,
 1.0,
 0.5555555555555556,
 0.4444444444444444,
 0.2222222222222222,
 0.3333333333333333,
 0.5555555555555556,
 0.7777777777777778,
 0.0,
 1.0,
 0.8888888888888888,
 0.5555555555555556,
 0.0,
 0.0,
 0.8888888888888888,
 0.6666666666666666,
 0.8888888888888888,
 0.7777777777777778,
 1.0,
 0.0,
 0.7777777777777778,
 0.8888888888888888,
 0.5555555555555556,
 0.3333333333333333,
 0.2222222222222222,
 0.6666666666666666,
 1.0,
 1.0,
 0.8888888888888888,
 0.4444444444444444,
 0.6666666666666666,
 0.6666666666666666,
 0.5555555555555556,
 0.6666666666666666,
 0.4444444444444444,
 0.0,
 0.5555555555555556,
 0.6666666666666666,
 0.4444444444444444,
 0.0,
 0.3333333333333333,
 1.0,
 0.4444444444444444,
 0.5555

In [48]:
# so given: parent genes, generations, time. predict soldprice
sale =  sales['sales'][600]
kid = sale['returnValues']['tokenId']
genetics[kid]
api_data[kid]


{'auction': {},
 'bio': 'Hey cutie! I\'m G0 Progenitor. My friends describe me as stinky and raunchy. My friends describe me as "tantalizing." It\'s... accurate. Let\'s pawty!',
 'cattributes': ['strawberry',
  'luckystripe',
  'sphynx',
  'soserious',
  'granitegrey',
  'chocolate',
  'thicccbrowz',
  'greymatter'],
 'children': [{'color': 'strawberry',
   'created_at': '2017-11-25T12:52:20.000Z',
   'generation': 13,
   'id': 7003,
   'image_url': 'https://storage.googleapis.com/ck-kitty-image/0x06012c8cf97bead5deae237070f9587f8e7a266d/7003.svg',
   'is_fancy': False,
   'name': 'Tooe Gaen',
   'owner_wallet_address': '0x118c191228cc79cfc886086fc346d1d47e29c2b1',
   'status': {'cooldown': 1512873963295,
    'is_gestating': False,
    'is_ready': False}},
  {'color': 'strawberry',
   'created_at': '2017-11-26T07:39:10.000Z',
   'generation': 13,
   'id': 8056,
   'image_url': 'https://storage.googleapis.com/ck-kitty-image/0x06012c8cf97bead5deae237070f9587f8e7a266d/8056.svg',
   'is_fa

# Train

Predict sale price given parents